In [65]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [66]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data["vehicle_capacities"] = [25, 25, 30, 20, 20, 30]
    data["num_vehicles"] = 6
    data["depot"] = 0

    data["pickups_deliveries"] = [
        [1, 6],
    ]
    data["restricted_roads"] = [(5, 8), (5, 8)]
    data["restricted_locations"] = {3: [1]}
    data["vehicle_restricted_roads"] = {
        0: [(3, 4)],  # Vehicle 0 should avoid edge 3 -> 4
        1: [(1, 2), (2, 1)],  # Vehicle 1 should avoid edges 1 -> 2 and 2 -> 1
    }
    return data

In [67]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_capacity = data["vehicle_capacities"][vehicle_id]
        plan_output = (
            f"Route for vehicle {vehicle_id} (Max capacity {vehicle_capacity}):\n"
        )
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

In [68]:
# Instantiate the data problem.
data = create_data_model()

In [69]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

In [70]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [71]:
# Define cost of each arc for each vehicle.
def create_vehicle_cost_callback(vehicle_id, data, manager):
    """Returns a transit callback that accounts for vehicle-specific and general restrictions."""

    def transit_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Check if the road is restricted specifically for the given vehicle
        if (from_node, to_node) in data.get("vehicle_restricted_roads", {}).get(
            vehicle_id, []
        ):
            return int(1e6)  # High cost to avoid this edge for this vehicle

        # Check if the road is restricted for all vehicles
        if (from_node, to_node) in data["restricted_roads"]:
            return int(1e6)  # High cost to avoid this edge for all vehicles
        # Default distance from the distance matrix
        vehicle_capacity = data["vehicle_capacities"][vehicle_id]
        distance = data["distance_matrix"][from_node][to_node]
        return distance

    return transit_callback

In [72]:
# Register vehicle-specific transit callbacks
for vehicle_id in range(data["num_vehicles"]):
    transit_callback = create_vehicle_cost_callback(vehicle_id, data, manager)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    routing.SetArcCostEvaluatorOfVehicle(transit_callback_index, vehicle_id)

In [73]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]


demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

True

In [74]:
# Add Distance constraint.
dimension_name = "Distance"
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    3000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [75]:
# Define Transportation Requests.
for request in data["pickups_deliveries"]:
    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.solver().Add(
        routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
    )

In [76]:
# Avoid specific locations for specific vehicles
restricted_locations = data["restricted_locations"]  # {vehicle_id: [location_ids]}
for vehicle_id, locations in restricted_locations.items():
    for location in locations:
        index = manager.NodeToIndex(location)
        routing.solver().Add(routing.VehicleVar(index) != vehicle_id)

In [77]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
)

In [78]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [79]:
if solution:
    print_solution(data, manager, routing, solution)

Objective: 185760
Route for vehicle 0 (Max capacity 25):
 0 Load(0) ->  7 Load(8) ->  1 Load(9) ->  6 Load(13) ->  5 Load(15) ->  0 Load(15)
Distance of the route: 1780m
Load of the route: 15

Route for vehicle 1 (Max capacity 25):
 0 Load(0) ->  10 Load(2) ->  2 Load(3) ->  8 Load(11) ->  0 Load(11)
Distance of the route: 1712m
Load of the route: 11

Route for vehicle 2 (Max capacity 30):
 0 Load(0) ->  9 Load(1) ->  14 Load(5) ->  16 Load(13) ->  0 Load(13)
Distance of the route: 1324m
Load of the route: 13

Route for vehicle 3 (Max capacity 20):
 0 Load(0) ->  13 Load(4) ->  15 Load(12) ->  11 Load(13) ->  12 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 4 (Max capacity 20):
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  0 Load(6)
Distance of the route: 1392m
Load of the route: 6

Route for vehicle 5 (Max capacity 30):
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Total distance of all routes: 7760m
Total load of